# Listener Notebook

In [8]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

from helpers.listener import (
    fetch_options_contracts,
    fetch_options_symbols,
    get_expiration_dates,
    get_liquid_options,
    get_options_chain,
    get_strikes,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Get 10 NVDA option symbols
symbols = fetch_options_symbols(["NVDA"], limit_per_ticker=1000)

# Get full contract details
contracts = fetch_options_contracts(["AAPL", "TSLA"], contract_type="call")

# Find expirations
dates = get_expiration_dates("NVDA", days_ahead=30)

# Find strikes for a specific expiration
strikes = get_strikes("NVDA", dates[0])
print(strikes)

  NVDA: 1000 contracts

Total: 1000 option symbols
  AAPL: 100 contracts
  TSLA: 100 contracts

Total: 200 contracts
[50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 152.5, 155, 157.5, 160, 162.5, 165, 167.5, 170, 172.5, 175, 177.5, 180, 182.5, 185, 187.5, 190, 192.5, 195, 197.5, 200, 202.5, 205, 207.5, 210, 212.5, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 320, 330, 340, 350, 360, 370, 380]


In [6]:
print(symbols)

['O:NVDA260102C00050000', 'O:NVDA260102C00055000', 'O:NVDA260102C00060000', 'O:NVDA260102C00065000', 'O:NVDA260102C00070000', 'O:NVDA260102C00075000', 'O:NVDA260102C00080000', 'O:NVDA260102C00085000', 'O:NVDA260102C00090000', 'O:NVDA260102C00095000', 'O:NVDA260102C00100000', 'O:NVDA260102C00105000', 'O:NVDA260102C00110000', 'O:NVDA260102C00115000', 'O:NVDA260102C00120000', 'O:NVDA260102C00125000', 'O:NVDA260102C00130000', 'O:NVDA260102C00135000', 'O:NVDA260102C00140000', 'O:NVDA260102C00145000', 'O:NVDA260102C00150000', 'O:NVDA260102C00152500', 'O:NVDA260102C00155000', 'O:NVDA260102C00157500', 'O:NVDA260102C00160000', 'O:NVDA260102C00162500', 'O:NVDA260102C00165000', 'O:NVDA260102C00167500', 'O:NVDA260102C00170000', 'O:NVDA260102C00172500', 'O:NVDA260102C00175000', 'O:NVDA260102C00177500', 'O:NVDA260102C00180000', 'O:NVDA260102C00182500', 'O:NVDA260102C00185000', 'O:NVDA260102C00187500', 'O:NVDA260102C00190000', 'O:NVDA260102C00192500', 'O:NVDA260102C00195000', 'O:NVDA260102C00197500',

In [24]:
print(dates)

['2026-01-02', '2026-01-09', '2026-01-16', '2026-01-23', '2026-01-30']


In [26]:
print(len(strikes))

272


In [2]:
# Get full chain with liquidity metrics
chain = get_options_chain("NVDA")

# Filter to find liquid options only
chain = get_options_chain("NVDA", min_volume=100, min_open_interest=500, max_spread_pct=0.10)

# Get just the symbols for liquid options
symbols = get_liquid_options("NVDA")  # Uses default filters

Fetched 3568 contracts, 3568 passed filters
Fetched 3568 contracts, 0 passed filters
Fetched 3568 contracts, 0 passed filters


In [12]:
# Get NVDA chain with liquidity data for a specific expiration
chain = get_options_chain("NVDA")
# Filter to your target expiration
target_exp = dates[3]  # e.g., "2026-01-02"
exp_chain = [c for c in chain if str(c["expiration"]) == str(target_exp)]
# Group by strike, sum volume + OI for calls and puts
strike_liquidity = defaultdict(lambda: {"volume": 0, "oi": 0, "contracts": 0})
for c in exp_chain:
    strike = c["strike"]
    strike_liquidity[strike]["volume"] += c["volume"] or 0
    strike_liquidity[strike]["oi"] += c["open_interest"] or 0
    strike_liquidity[strike]["contracts"] += 1
# Sort by total open interest (most liquid first)
liquid_strikes = sorted(strike_liquidity.items(), key=lambda x: x[1]["oi"], reverse=True)

Fetched 3568 contracts, 3568 passed filters


In [14]:
# Get all NVDA expirations (next 30 days)
dates = get_expiration_dates("NVDA", days_ahead=30)
# Get full chain once (faster than multiple API calls)
chain = get_options_chain("NVDA")
# Define "liquid" thresholds
MIN_OI = 500
MIN_VOLUME = 100
print(f"Liquidity by expiration (OI >= {MIN_OI} or Volume >= {MIN_VOLUME})")
print(f"{'Expiration':<12} {'Liquid':>8} {'Total':>8} {'Pct':>8}")
print("-" * 40)
for exp_date in dates:
    # Filter chain to this expiration
    exp_chain = [c for c in chain if str(c["expiration"]) == str(exp_date)]

    # Get unique strikes
    all_strikes = set(c["strike"] for c in exp_chain)

    # Find liquid strikes (either call or put has volume/OI)
    liquid_strikes = set()
    for c in exp_chain:
        if (c["open_interest"] or 0) >= MIN_OI or (c["volume"] or 0) >= MIN_VOLUME:
            liquid_strikes.add(c["strike"])

    pct = len(liquid_strikes) / len(all_strikes) * 100 if all_strikes else 0
    print(f"{exp_date}   {len(liquid_strikes):>8} / {len(all_strikes):>6}   {pct:>6.1f}%")

Fetched 3568 contracts, 3568 passed filters
Liquidity by expiration (OI >= 500 or Volume >= 100)
Expiration     Liquid    Total      Pct
----------------------------------------
2026-01-02         62 /     73     84.9%
2026-01-09         54 /     70     77.1%
2026-01-16        271 /    272     99.6%
2026-01-23         39 /     58     67.2%
2026-01-30         33 /     58     56.9%


In [17]:
print(exp_date)
print(sorted(liquid_strikes))

2026-01-30
[80, 100, 105, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 245, 250, 260, 270, 295, 305, 320]
